In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [8]:
client_id = '89b79519edb2463ca8d11e52a2ba9cbb'
client_secret = 'f78401a575fd4bb7bad5378aed900c93'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# Fetch artists and top tracks
def fetch_artists_tracks(genre='country', limit=50):
    artist_tracks = {}
    results = sp.search(q='genre:' + genre, type='artist', limit=limit)
    
    for item in results['artists']['items']:
        artist_id = item['id']
        artist_name = item['name']
        top_tracks = sp.artist_top_tracks(artist_id)
        
        for track in top_tracks['tracks']:
            for artist in track['artists']:
                if artist['name'] != artist_name:
                    artist_tracks.setdefault(artist_name, set()).add(artist['name'])
    return artist_tracks

In [4]:
# Create and build the graph
def build_graph(artist_tracks):
    G = nx.Graph()
    for artist, collaborators in artist_tracks.items():
        for collaborator in collaborators:
            G.add_edge(artist, collaborator, weight=G[artist][collaborator]['weight'] + 1 if G.has_edge(artist, collaborator) else 1)
    return G

In [5]:
def count_collaborations(G):
    return {artist: sum(data['weight'] for _, data in G[artist].items()) for artist in G.nodes()}

In [6]:
# Fetch artist tracks and build the graph
artist_tracks = fetch_artists_tracks()
G = build_graph(artist_tracks)

# Count collaborations and sort
collaboration_counts = count_collaborations(G)
sorted_collaborations = sorted(collaboration_counts.items(), key=lambda x: x[1], reverse=True)

In [9]:
# Print top 10 influential artists
print("Top 10 Influential Artists:")
for artist, count in sorted_collaborations[:10]:
    print(f"{artist}:")
    print(f"  Number of Collaborations: {count}")
    print(f"  Degree Centrality: {nx.degree_centrality(G)[artist]}\n")

Top 10 Influential Artists:
Willie Nelson:
  Number of Collaborations: 27
  Degree Centrality: 0.20689655172413793

Eric Church:
  Number of Collaborations: 21
  Degree Centrality: 0.1724137931034483

Post Malone:
  Number of Collaborations: 10
  Degree Centrality: 0.08620689655172414

Jelly Roll:
  Number of Collaborations: 9
  Degree Centrality: 0.06034482758620689

Jessie Murph:
  Number of Collaborations: 9
  Degree Centrality: 0.05172413793103448

Morgan Wallen:
  Number of Collaborations: 8
  Degree Centrality: 0.06034482758620689

HARDY:
  Number of Collaborations: 8
  Degree Centrality: 0.06034482758620689

Johnny Cash:
  Number of Collaborations: 8
  Degree Centrality: 0.06034482758620689

Lainey Wilson:
  Number of Collaborations: 7
  Degree Centrality: 0.04310344827586207

Shaboozey:
  Number of Collaborations: 7
  Degree Centrality: 0.06034482758620689



In [13]:
# Export the graph to a GraphML file
#nx.write_graphml(G, 'spotify_network.graphml')